In [240]:
import datetime
import json
#import urllib.request
#from urllib.request import urlopen, Request
from urllib2 import urlopen

def time_converter(time):
    converted_time = datetime.datetime.fromtimestamp(
        int(time)
    ).strftime('%I:%M %p')
    return converted_time


def url_builder(city_id,selector='forecast',start_date='',end_date=''):
    user_api = '790eed0a098e44fbb986d3e372ca3a2b'  # Obtain yours form: http://openweathermap.org/
    
    if selector == 'forecast':
        api = 'http://api.weatherbit.io/v2.0/forecast/3hourly?city_id='     # Search for your city ID here: http://bulk.openweathermap.org/sample/city.list.json.gz
        full_api_url = api + str(city_id) + '&key=' + user_api

    if selector == 'history':
        api = 'http://api.weatherbit.io/v2.0/history/hourly?city_id='     # Search for your city ID here: http://bulk.openweathermap.org/sample/city.list.json.gz
        full_api_url = api + str(city_id) + '&key=' + user_api
        full_api_url += '&start_date=' + start_date + '&end_date=' + end_date

    return full_api_url


def data_fetch(full_api_url):
#    url = urllib.request.urlopen(full_api_url)
    print(full_api_url)

    url = urlopen(full_api_url)
 
    output = url.read().decode('utf-8')
    raw_api_dict = json.loads(output)
    url.close()
    return raw_api_dict


def data_organizer(raw_api_dict):
    data = dict(
        city=raw_api_dict.get('name'),
        country=raw_api_dict.get('sys').get('country'),
        temp=raw_api_dict.get('main').get('temp'),
        temp_max=raw_api_dict.get('main').get('temp_max'),
        temp_min=raw_api_dict.get('main').get('temp_min'),
        humidity=raw_api_dict.get('main').get('humidity'),
        pressure=raw_api_dict.get('main').get('pressure'),
        sky=raw_api_dict['weather'][0]['main'],
        sunrise=time_converter(raw_api_dict.get('sys').get('sunrise')),
        sunset=time_converter(raw_api_dict.get('sys').get('sunset')),
        wind=raw_api_dict.get('wind').get('speed'),
        wind_deg=raw_api_dict.get('deg'),
        dt=time_converter(raw_api_dict.get('dt')),
        cloudiness=raw_api_dict.get('clouds').get('all')
    )
    return data

#if __name__ == '__main__':
#    try:
#        data_output(data_organizer(data_fetch(url_builder(2026609))))
#        url = url_builder(2026609)
#

#        
#    
#    except IOError:
#        print('no internet')

In [228]:
import pandas
import datetime

def process_data(fetched_data):

    df = pandas.DataFrame(columns=['date','LT','dt_txt','T','U'])
    i = 1

    for box in fetched_data['data']:

        date = datetime.datetime.fromtimestamp(int(box['ts'])) + datetime.timedelta(hours=9, minutes=0)
        df.loc[len(df)] = date.date(),date,'',box['temp'],box['rh']

    df.set_index('date', inplace=True)

    return df

def process_flatten(df):

    ds0600 = df[df.LT.dt.hour == 6]
    ds0900 = df[df.LT.dt.hour == 9]
    ds1200 = df[df.LT.dt.hour == 12]
    ds1500 = df[df.LT.dt.hour == 15]
    ds1800 = df[df.LT.dt.hour == 18]
    ds2100 = df[df.LT.dt.hour == 21]

    ds0912 = ds0900.join(ds1200, lsuffix="09", rsuffix="12",how = 'outer' )
    ds1518 = ds1500.join(ds1800, lsuffix="15", rsuffix="18",how = 'outer' )
    ds0621 = ds0600.join(ds2100, lsuffix="06", rsuffix="21",how = 'outer' )

    ds09121518 = ds0912.join(ds1518,how='outer')
    ds091215180621 = ds09121518.join(ds0621,how='outer')

    ds091215180621.drop(['LT09','LT12','LT15','LT18','LT06','LT21'], axis=1,inplace=True)
    ds091215180621.drop(['dt_txt09','dt_txt12','dt_txt15','dt_txt18','dt_txt06','dt_txt21'], axis=1,inplace=True)

#    ds091215180621['T1330'] = ds091215180621[['T12','T15']].mean(axis=1)
#    ds091215180621['U1330'] = ds091215180621[['U12','U15']].mean(axis=1)
#    ds091215180621.drop(['T12','U12','T15','U15'], axis=1,inplace=True)
    ds091215180621.drop(['T06','U06'], axis=1,inplace=True)

    return ds091215180621

In [229]:
#history = pandas.DataFrame(columns=['date','T09','U09','T18','U18','T21','U21','T1330','U1330'])
history = pandas.DataFrame(columns=['date','T09','U09','T12','U12','T15','U15','T18','U18','T21','U21'])
history.set_index('date', inplace=True)

first = True

for i in range (-4,1):
    dt = datetime.datetime.utcnow().date()
    start_dt = (dt + datetime.timedelta(days=i,hours=0, minutes=0)).strftime("%Y-%m-%d")
    end_dt = (dt + datetime.timedelta(days=i+1,hours=0, minutes=0)).strftime("%Y-%m-%d")

    url = url_builder(2036973, selector = 'history', start_date = start_dt, end_date = end_dt)
    fetched_data = data_fetch(url)
    
    if first == True:
        df = process_data(fetched_data)
        first = False
    else:
        df = df.append(process_data(fetched_data))
#    df = df.append(process_flatten(df))

df.drop_duplicates(keep='first',inplace=True)

history = process_flatten(df)

http://api.weatherbit.io/v2.0/history/hourly?city_id=2036973&key=790eed0a098e44fbb986d3e372ca3a2b&start_date=2018-04-23&end_date=2018-04-24
http://api.weatherbit.io/v2.0/history/hourly?city_id=2036973&key=790eed0a098e44fbb986d3e372ca3a2b&start_date=2018-04-24&end_date=2018-04-25
http://api.weatherbit.io/v2.0/history/hourly?city_id=2036973&key=790eed0a098e44fbb986d3e372ca3a2b&start_date=2018-04-25&end_date=2018-04-26
http://api.weatherbit.io/v2.0/history/hourly?city_id=2036973&key=790eed0a098e44fbb986d3e372ca3a2b&start_date=2018-04-26&end_date=2018-04-27
http://api.weatherbit.io/v2.0/history/hourly?city_id=2036973&key=790eed0a098e44fbb986d3e372ca3a2b&start_date=2018-04-27&end_date=2018-04-28


In [230]:

history

,T09,U09,T12,U12,T15,U15,T18,U18,T21,U21
date,,,,,,,,,,
2018-04-23,10.2,24.3333,18.6,14.3333,24,8.66667,23.1,9,17.6,14.6667
2018-04-24,18.3,14.3333,24.7,8.66667,25.2,9,25.1,9,17.8,16.3333
2018-04-25,20.4,17.6667,25.6,11.6667,27.7,10.6667,25.3,13,12.9,42
2018-04-26,5,25,7,14,9,14,8,12,1,22
2018-04-27,8,15,12,11,15,11,14,14,10,18


In [231]:
url = url_builder(2036973, selector = 'forecast')
fetched_data = data_fetch(url)

forecast = process_flatten(process_data(fetched_data))

http://api.weatherbit.io/v2.0/forecast/3hourly?city_id=2036973&key=790eed0a098e44fbb986d3e372ca3a2b


In [232]:
forecast

,T09,U09,T12,U12,T15,U15,T18,U18,T21,U21
date,,,,,,,,,,
2018-04-28,13.6,26,17.5,21,20.1,18,19.2,23,15.6,29
2018-04-29,8.7,40,13.4,23,14.4,20,13.2,19,8.9,27
2018-04-30,7.2,24,10.4,17,12.9,15,13.2,14,8.8,22
2018-05-01,10.0,29,13.8,21,14.9,18,14.8,18,11.3,24
2018-05-02,14.0,25,18.9,17,20.3,14,20.4,16,14.9,24


In [233]:
forecast.to_csv('forecast-' + str(datetime.datetime.now()) + '.csv')
history.to_csv('historical-' + str(datetime.datetime.now()) + '.csv')

In [234]:
pandas.read_csv('forecast-2018-04-27 07:56:48.113775.csv')

,date,T09,U09,T12,U12,T15,U15,T18,U18,T21,U21
0,2018-04-27,NaN,NaN,NaN,NaN,14.1,18.0,14.2,19.0,9.6,27.0
1,2018-04-28,13.9,25.0,17.9,20.0,20.6,18.0,20.0,22.0,10.7,50.0
2,2018-04-29,8.6,39.0,12.7,24.0,14.1,20.0,13.3,20.0,10.4,25.0
3,2018-04-30,7.0,28.0,9.8,19.0,11.4,16.0,11.7,16.0,7.7,21.0
4,2018-05-01,11.9,28.0,14.8,17.0,15.2,19.0,15.1,21.0,11.5,28.0
5,2018-05-02,14.5,27.0,19.4,18.0,NaN,NaN,NaN,NaN,NaN,NaN


In [235]:
import sqlite3
import pandas as pd
# Create your connection.
cnx = sqlite3.connect('file.db')

#df = pd.read_sql_query("SELECT * FROM table_name", cnx)

forecast["forecast_datetime"] = datetime.datetime.now()
forecast.to_sql("forecast", cnx, if_exists="append")

history.to_sql("history", cnx, if_exists="append")

cnx.close()



In [236]:
forecast

,T09,U09,T12,U12,T15,U15,T18,U18,T21,U21,forecast_datetime
date,,,,,,,,,,,
2018-04-28,13.6,26,17.5,21,20.1,18,19.2,23,15.6,29,2018-04-27 15:14:54.717174
2018-04-29,8.7,40,13.4,23,14.4,20,13.2,19,8.9,27,2018-04-27 15:14:54.717174
2018-04-30,7.2,24,10.4,17,12.9,15,13.2,14,8.8,22,2018-04-27 15:14:54.717174
2018-05-01,10.0,29,13.8,21,14.9,18,14.8,18,11.3,24,2018-04-27 15:14:54.717174
2018-05-02,14.0,25,18.9,17,20.3,14,20.4,16,14.9,24,2018-04-27 15:14:54.717174


In [237]:
cnx = sqlite3.connect('file.db')



In [239]:
pd.read_sql_query("SELECT * FROM forecast WHERE forecast_datetime = (SELECT MAX(forecast_datetime) FROM forecast)", cnx)

,forecast_datetime,date,T09,U09,T12,U12,T15,U15,T18,U18,T21,U21
0,2018-04-27 15:14:54.717174,2018-04-28,13.6,26,17.5,21,20.1,18,19.2,23,15.6,29
1,2018-04-27 15:14:54.717174,2018-04-29,8.7,40,13.4,23,14.4,20,13.2,19,8.9,27
2,2018-04-27 15:14:54.717174,2018-04-30,7.2,24,10.4,17,12.9,15,13.2,14,8.8,22
3,2018-04-27 15:14:54.717174,2018-05-01,10.0,29,13.8,21,14.9,18,14.8,18,11.3,24
4,2018-04-27 15:14:54.717174,2018-05-02,14.0,25,18.9,17,20.3,14,20.4,16,14.9,24


In [217]:
pd.read_sql_query("SELECT * FROM history", cnx)

,date,T09,U09,T12,U12,T15,U15,T18,U18,T21,U21
0,2018-04-23,10.2,24.3333,18.7,14.66670,22.8,9.66667,23.1,9.0,17.6,14.6667
1,2018-04-24,18.3,14.3333,24.7,8.66667,25.2,9.00000,25.1,9.0,17.8,16.3333
2,2018-04-25,22.0,13.0000,27.0,10.00000,28.0,8.00000,19.0,25.0,12.0,40.0000
3,2018-04-26,5.0,25.0000,7.0,14.00000,9.0,14.00000,8.0,12.0,1.0,22.0000
4,2018-04-27,8.0,15.0000,12.0,11.00000,15.0,11.00000,NaN,NaN,NaN,NaN
5,2018-04-23,10.2,24.3333,18.7,14.66670,22.8,9.66667,23.1,9.0,17.6,14.6667
6,2018-04-24,18.3,14.3333,24.7,8.66667,25.2,9.00000,25.1,9.0,17.8,16.3333
7,2018-04-25,22.0,13.0000,27.0,10.00000,28.0,8.00000,19.0,25.0,12.0,40.0000
8,2018-04-26,5.0,25.0000,7.0,14.00000,9.0,14.00000,8.0,12.0,1.0,22.0000
9,2018-04-27,8.0,15.0000,12.0,11.00000,15.0,11.00000,NaN,NaN,NaN,NaN


In [219]:
cnx = sqlite3.connect('file.db')

#res = cnx.execute("SELECT name FROM sqlite_master WHERE type='table';")
res = cnx.execute("select sql from sqlite_master where name='forecast';")

for name in res:
    print name[0]

CREATE TABLE "forecast" ( "forecast_datetime" DATETIME, "date" DATE, "T09" REAL, "U09" INTEGER, "T12" REAL, "U12" INTEGER, "T15" REAL, "U15" INTEGER, "T18" REAL, "U18" INTEGER, "T21" REAL, "U21" INTEGER )


In [201]:
type(test.T09[0])

unicode

In [200]:
type(history.T09[0])

float

In [218]:
#res = cnx.execute("drop table 'forecast';")
#res = cnx.execute('CREATE TABLE "forecast" ( "forecast_datetime" DATETIME, "date" DATE, "T09" REAL, "U09" INTEGER, "T12" REAL, "U12" INTEGER, "T15" REAL, "U15" INTEGER, "T18" REAL, "U18" INTEGER, "T21" REAL, "U21" INTEGER );')